created on: Fri Jan 13 14:20:37 2020
<br>
Group 7
<br>
@authors: E.G., C.L.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import math
#import operator
import random
import os
from PIL import Image
from shutil import copyfile
import shutil
import pickle

# deep learning
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
%run g7_functions_for_models.ipynb

In [3]:
project_path = './../'
seatguru_path = './Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'

In [4]:
# Read annotated CSV
df_seat_annot = pd.read_csv('./../CSV_annotate/SEATGURU/g7_SEATGURU_annotate.csv', sep=';', engine='python',
                            index_col=None, encoding='utf-8')


In [5]:
airbus_planes = ['A320', 'A321', 'A330', 'A350', 'A380']
crea_path = project_path + 'G7_SEATGURU/Int/Airbus/'
new_paths = [crea_path + 'data_train', crea_path + 'data_test']
print(crea_path + 'data_train')

# Create a directory for each class
create_dirs_seatguru_type(df_seat_annot, project_path, seatguru_path, crea_path, aircraft_types=airbus_planes,
                          view = 'Int', man = 'Airbus')

# Split train and test
split_train_test_seatguru_type(new_paths=new_paths, path=crea_path, aircraft_types=airbus_planes)

./../G7_SEATGURU/Int/Airbus/data_train
./../G7_SEATGURU/Int/Airbus/A320
A320: 166 images
./../G7_SEATGURU/Int/Airbus/A321
A321: 121 images
./../G7_SEATGURU/Int/Airbus/A330
A330: 257 images
./../G7_SEATGURU/Int/Airbus/A350
A350: 39 images
./../G7_SEATGURU/Int/Airbus/A380
A380: 46 images


In [6]:
# Image data generator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(new_paths[0],
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(new_paths[1],
                                                   target_size=(224,224),
                                                   color_mode='rgb',
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   shuffle=True)

Found 438 images belonging to 5 classes.
Found 191 images belonging to 5 classes.


# Transfer learning

In [19]:
# Create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add layers
x = base_model.output

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Last layer used to predict our classes
# Dense as the same number of neurons as the number of classes to predict

predictions = Dense(5, activation = 'softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [20]:
# Reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

In [21]:
step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=20,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])

Epoch 1/20
13/13 [==============================] - 97s 7s/step - loss: 1.8681 - accuracy: 0.3571 - val_loss: 10.9541 - val_accuracy: 0.2042
Epoch 2/20
13/13 [==============================] - 97s 7s/step - loss: 0.5229 - accuracy: 0.8325 - val_loss: 4.6073 - val_accuracy: 0.2880
Epoch 3/20
13/13 [==============================] - 99s 8s/step - loss: 0.2135 - accuracy: 0.9384 - val_loss: 3.2599 - val_accuracy: 0.3141
Epoch 4/20
13/13 [==============================] - 102s 8s/step - loss: 0.0614 - accuracy: 0.9904 - val_loss: 3.8952 - val_accuracy: 0.3717
Epoch 5/20
13/13 [==============================] - 97s 7s/step - loss: 0.0239 - accuracy: 0.9949 - val_loss: 2.6477 - val_accuracy: 0.3403
Epoch 6/20
13/13 [==============================] - 99s 8s/step - loss: 0.0224 - accuracy: 0.9928 - val_loss: 2.0423 - val_accuracy: 0.3927
Epoch 7/20
13/13 [==============================] - 103s 8s/step - loss: 0.0203 - accuracy: 0.9975 - val_loss: 2.2430 - val_accuracy: 0.3874
Epoch 8/20
13/13 

In [ ]:
path_mod = './../Models/'
save_model_classes(path_mod, 'Int_Airbus', train_generator, model)